# Getting the relation between baroclinic streamfunction and cross-stream distance from the Argo data

In [1]:
import pickle
import numpy as np
import xarray as xr
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import interp1d

import gsw
import csaps

import warnings

## Sample points along hydrographic section transect

In [2]:
# read in hydrographic section data
data = loadmat('data/SR1b_section/sr1b_all.mat')

icyc = np.where(data['buf_year'] != 1999)[1] # exclude 1999 # TODO: why?

buf_lat = data['buf_lat'][0,icyc]
buf_lon = data['buf_lon'][0,icyc]
buf_sal = data['buf_sal'][:,icyc]
buf_temp = data['buf_temp'][:,icyc]
buf_press = data['buf_press'].reshape(3000)
buf_year = data['buf_year'][:,icyc].reshape(316)

In [3]:
# sample points along section
start_lon_vals = []
end_lon_vals = []
start_lat_vals = []
end_lat_vals = []
for yr in np.unique(buf_year):
  idx_yr = np.where(buf_year == yr)[0]
  start_lon_vals.append(np.min(buf_lon[idx_yr]))
  end_lon_vals.append(np.max(buf_lon[idx_yr]))
  start_lat_vals.append(np.max(buf_lat[idx_yr]))
  end_lat_vals.append(np.min(buf_lat[idx_yr]))
start_lon = np.mean(np.array(start_lon_vals))
end_lon = np.mean(np.array(end_lon_vals))
start_lat = np.mean(np.array(start_lat_vals))
end_lat = np.mean(np.array(end_lat_vals))

# Extract unique points along the section
nr_points = 30
lon_along_section = np.linspace(start_lon, end_lon, nr_points)
lat_along_section = np.linspace(start_lat, end_lat, nr_points)

# calculate distance from southern end of section
dist_along_section = np.zeros(len(lat_along_section))
for i in range(len(dist_along_section)):
  dist_along_section[i] = gsw.distance(np.mean(lon_along_section)*np.ones(2), np.array([min(lat_along_section), lat_along_section[i]]))[0]

## Process the Argo data

In [4]:
argo_salt_dataset = xr.open_dataset('data/Argo_data/RG_ArgoClim_Salinity_2019.nc',decode_times=False)
argo_temp_dataset = xr.open_dataset('data/Argo_data/RG_ArgoClim_Temperature_2019.nc',decode_times=False)
# downloaded from https://sio-argo.ucsd.edu/RG_Climatology.html on 14 July 2025

argo_salt_mean_all = xr.DataArray(argo_salt_dataset['ARGO_SALINITY_MEAN'].values,
                                  dims = ['PRESSURE', 'LATITUDE', 'LONGITUDE'],
                                  coords = {'PRESSURE': argo_salt_dataset['PRESSURE'].values,
                                        'LATITUDE': argo_salt_dataset['LATITUDE'].values,
                                        'LONGITUDE': argo_salt_dataset['LONGITUDE'].values})
argo_salt_anom_all = xr.DataArray(argo_salt_dataset['ARGO_SALINITY_ANOMALY'].values,
                                  dims = ['TIME', 'PRESSURE', 'LATITUDE', 'LONGITUDE'],
                                  coords = {'TIME': argo_salt_dataset['TIME'].values,
                                        'PRESSURE': argo_salt_dataset['PRESSURE'].values,
                                        'LATITUDE': argo_salt_dataset['LATITUDE'].values,
                                        'LONGITUDE': argo_salt_dataset['LONGITUDE'].values})

argo_temp_mean_all = xr.DataArray(argo_temp_dataset['ARGO_TEMPERATURE_MEAN'].values,
                                  dims = ['PRESSURE', 'LATITUDE', 'LONGITUDE'],
                                  coords = {'PRESSURE': argo_temp_dataset['PRESSURE'].values,
                                        'LATITUDE': argo_temp_dataset['LATITUDE'].values,
                                        'LONGITUDE': argo_temp_dataset['LONGITUDE'].values})
argo_temp_anom_all = xr.DataArray(argo_temp_dataset['ARGO_TEMPERATURE_ANOMALY'].values,
                                  dims = ['TIME', 'PRESSURE', 'LATITUDE', 'LONGITUDE'],
                                  coords = {'TIME': argo_temp_dataset['TIME'].values,
                                        'PRESSURE': argo_temp_dataset['PRESSURE'].values,
                                        'LATITUDE': argo_temp_dataset['LATITUDE'].values,
                                        'LONGITUDE': argo_temp_dataset['LONGITUDE'].values})

# select region of interest
argo_salt_mean = argo_salt_mean_all.sel(LATITUDE=slice(-62, -53), LONGITUDE=slice(300, 307))
argo_salt_anom = argo_salt_anom_all.sel(LATITUDE=slice(-62, -53), LONGITUDE=slice(300, 307))
argo_salinity = argo_salt_anom + argo_salt_mean

argo_temp_mean = argo_temp_mean_all.sel(LATITUDE=slice(-62, -53), LONGITUDE=slice(300, 307))
argo_temp_anom = argo_temp_anom_all.sel(LATITUDE=slice(-62, -53), LONGITUDE=slice(300, 307))
argo_temperature = argo_temp_anom + argo_temp_mean

argo_time = argo_salinity['TIME'].values
argo_pressure = argo_salinity['PRESSURE'].values
argo_lat = argo_salinity['LATITUDE'].values
argo_lon = argo_salinity['LONGITUDE'].values - 360 # convert to -180 to 180 range

In [5]:
# calculate absolute salinity and conservative temperature
# argo_SA = np.zeros(np.shape(argo_salinity))
# for t in range(len(argo_time)):
#     for d in range(len(argo_pressure)):
#         for i in range(len(argo_lat)):
#             for j in range(len(argo_lon)):
#                 argo_SA[t,d,i,j] = gsw.SA_from_SP(argo_salinity[t,d,i,j], argo_pressure[d], argo_lon[j], argo_lat[i])
# np.save('data/Argo_data/Argo_SA.npy', argo_SA)
argo_SA = xr.DataArray(np.load('data/Argo_data/Argo_SA.npy'), 
                        dims=['TIME', 'PRESSURE', 'LATITUDE', 'LONGITUDE'],
                        coords={'TIME': argo_time,
                                'PRESSURE': argo_pressure,
                                'LATITUDE': argo_lat,
                                'LONGITUDE': argo_lon})
argo_CT = gsw.CT_from_pt(argo_SA, argo_temperature.values)

In [13]:
# interpolate SA and CT to section points
SA_section_argo = np.zeros((len(argo_time), len(argo_pressure), len(lat_along_section)))
CT_section_argo = np.zeros((len(argo_time), len(argo_pressure), len(lat_along_section)))
points = np.array([(lat, lon) for lat,lon in zip(lat_along_section,lon_along_section)])
for i in range(len(argo_time)):
    for j in range(len(argo_pressure)):
        f_SA_argo = RegularGridInterpolator((argo_lat, argo_lon), argo_SA.values[i,j,:,:])
        f_CT_argo = RegularGridInterpolator((argo_lat, argo_lon), argo_CT.values[i,j,:,:])
        SA_section_argo[i,j,:] = f_SA_argo(points)
        CT_section_argo[i,j,:] = f_CT_argo(points)

# interpolate in p (to high resolution)
SA_section_interp_argo = np.zeros((len(argo_time), len(buf_press), len(lat_along_section)))
CT_section_interp_argo = np.zeros((len(argo_time), len(buf_press), len(lat_along_section)))
for i in range(len(argo_time)):
    for j in range(len(lat_along_section)):
        f_SA_argo = interp1d(argo_pressure, SA_section_argo[i,:,j], bounds_error=False, fill_value=np.nan)
        f_CT_argo = interp1d(argo_pressure, CT_section_argo[i,:,j], bounds_error=False, fill_value=np.nan)
        SA_section_interp_argo[i,:,j] = f_SA_argo(buf_press)
        CT_section_interp_argo[i,:,j] = f_CT_argo(buf_press)

In [14]:
# compute baroclinic streamfunction
gpan_argo = np.zeros((len(argo_time), len(buf_press), len(lat_along_section)))
for i in range(len(argo_time)):
    gpan_argo[i,:,:] = -gsw.geo_strf_dyn_height(SA_section_interp_argo[i,:,:], CT_section_interp_argo[i,:,:], buf_press, 1500)

argo_phi1500 = abs(gpan_argo[:,249,:])

# make spline fit through relationship between phi1500 and distance
# dist_repeat = np.tile(dist_along_section, len(argo_time))
# argo_phi1500_flatten = argo_phi1500.flatten()
# pp_dist_argo = csaps.csaps(argo_phi1500_flatten[np.argsort(argo_phi1500_flatten)],
#                            dist_repeat[np.argsort(argo_phi1500_flatten)],
#                            smooth = 0.85)

# also do it for the subset 2010-2012
argo_phi1500_sub = argo_phi1500[6*12:8*12,:]  # select 2010-2012
argo_time_sub = argo_time[6*12:8*12]  # select 2010-2012
dist_repeat_sub = np.tile(dist_along_section, len(argo_time_sub))
argo_phi1500_flatten_sub = argo_phi1500_sub.flatten()
pp_dist_argo = csaps.csaps(argo_phi1500_flatten_sub[np.argsort(argo_phi1500_flatten_sub)],
                           dist_repeat_sub[np.argsort(argo_phi1500_flatten_sub)],
                           smooth = 0.85)

# save the spline fit
with open('data/Argo_data/pp_dist_argo.pkl', 'wb') as f:
    pickle.dump(pp_dist_argo, f)

/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/csaps/_sspumv.py:301: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = la.spsolve(a, b)
